In [1]:
import pandas as pd

# Creating mock data
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Sex': ['Female', 'Male', 'Male', 'Male', 'Female'],
    'AgeGroup': ['20-29', '30-39', '40-49', '20-29', '30-39'],
    'License': ['Yes', 'No', 'Yes', 'No', 'Yes'],
    'AnyWork': ['Yes', 'Yes', 'No', 'Yes', 'No'],

}
data['Name'].extend(['Frank', 'Grace', 'Hannah', 'Ian', 'Jack'])
data['Sex'].extend(['Male', 'Female', 'Female', 'Male', 'Male'])
data['AgeGroup'].extend(['40-49', '20-29', '30-39', '40-49', '20-29'])
data['License'].extend(['No', 'Yes', 'No', 'Yes', 'No'])
data['AnyWork'].extend(['No', 'Yes', 'Yes', 'No', 'Yes'])
# Creating DataFrame
df = pd.DataFrame(data)
df

,Name,Sex,AgeGroup,License,AnyWork
0,Alice,Female,20-29,Yes,Yes
1,Bob,Male,30-39,No,Yes
2,Charlie,Male,40-49,Yes,No
3,David,Male,20-29,No,Yes
4,Eva,Female,30-39,Yes,No
5,Frank,Male,40-49,No,No
6,Grace,Female,20-29,Yes,Yes
7,Hannah,Female,30-39,No,Yes
8,Ian,Male,40-49,Yes,No
9,Jack,Male,20-29,No,Yes


In [2]:
import polars as pl
df = pl.from_pandas(df)

In [4]:
gg = df.drop("Name").with_columns(pl.lit(2).alias("count"))
# gg = gg.group_by(pl.col(["Sex", "License", "AgeGroup"])).agg(pl.sum("count").alias("count"))
# aa = gg.pivot("Sex", index=["AgeGroup", "License"], values="count")

In [10]:
def convert_to_required_ILP_format(syn_count: pl.DataFrame, att:str, adjusted_atts: list[str], count_field:str="count") -> pl.DataFrame:
    """Convert the disagg data to the required format for ILP"""
    # group by and pivot the syn to have adjusted_atts as index and states in att as columns
    assert count_field in syn_count.columns
    gb_syn = syn_count.group_by(pl.col(adjusted_atts+[att])).agg(pl.sum(count_field).alias(count_field))
    ILP_formatted_syn = gb_syn.pivot(att, index=adjusted_atts, values=count_field)
    ILP_formatted_syn = ILP_formatted_syn.cast({x: pl.Int32 for x in ILP_formatted_syn.columns if x not in adjusted_atts})
    return ILP_formatted_syn

In [11]:
mm = convert_to_required_ILP_format(gg, "Sex", ["AgeGroup", "License"])

In [16]:
mm.with_columns(pl.when(pl.col(pl.Int32).is_not_null()).then(pl.lit(0)).otherwise(pl.col(pl.Int32)).name.suffix(""))

AgeGroup,License,Male,Female
str,str,i32,i32
"""40-49""","""Yes""",0,null
"""20-29""","""No""",0,null
"""30-39""","""No""",0,0
"""40-49""","""No""",0,null
"""20-29""","""Yes""",null,0
"""30-39""","""Yes""",null,0


In [164]:
mm = aa.unpivot(["Male", "Female"], index=["AgeGroup","License"], variable_name="Sex", value_name="count").filter(pl.col("count").is_not_null())

In [201]:

# mm.with_columns(pl.when(pl.col(pl.String)=="Male").then(pl.lit("j")).otherwise(pl.lit("k")))
# mm.with_columns(hhe=pl.when(pl.col(pl.String)=="Male").then(pl.lit("j")).otherwise(pl.lit("k")))
mm.with_columns(pl.col(pl.String).replace_strict(["Male"], ["j"], default=pl.col(pl.String)))

AgeGroup,License,Sex,count
str,str,str,i32
"""30-39""","""No""","""j""",2
"""20-29""","""No""","""j""",4
"""40-49""","""Yes""","""j""",4
"""40-49""","""No""","""j""",2
"""30-39""","""No""","""Female""",2
"""20-29""","""Yes""","""Female""",4
"""30-39""","""Yes""","""Female""",2


In [20]:
b = df.with_columns(pl.concat_list(["License", "AnyWork"]).alias("test"))
b

Name,Sex,AgeGroup,License,AnyWork,test
str,str,str,str,str,list[str]
"""Alice""","""Female""","""20-29""","""Yes""","""Yes""","[""Yes"", ""Yes""]"
"""Bob""","""Male""","""30-39""","""No""","""Yes""","[""No"", ""Yes""]"
"""Charlie""","""Male""","""40-49""","""Yes""","""No""","[""Yes"", ""No""]"
"""David""","""Male""","""20-29""","""No""","""Yes""","[""No"", ""Yes""]"
"""Eva""","""Female""","""30-39""","""Yes""","""No""","[""Yes"", ""No""]"
"""Frank""","""Male""","""40-49""","""No""","""No""","[""No"", ""No""]"
"""Grace""","""Female""","""20-29""","""Yes""","""Yes""","[""Yes"", ""Yes""]"
"""Hannah""","""Female""","""30-39""","""No""","""Yes""","[""No"", ""Yes""]"
"""Ian""","""Male""","""40-49""","""Yes""","""No""","[""Yes"", ""No""]"


In [44]:
a = b.group_by(["Sex", "AgeGroup"]).agg(pl.col("test"), pl.len())
a

Sex,AgeGroup,test,len
str,str,list[list[str]],u32
"""Male""","""30-39""","[[""No"", ""Yes""]]",1
"""Female""","""30-39""","[[""Yes"", ""No""], [""No"", ""Yes""]]",2
"""Male""","""20-29""","[[""No"", ""Yes""], [""No"", ""Yes""]]",2
"""Female""","""20-29""","[[""Yes"", ""Yes""], [""Yes"", ""Yes""]]",2
"""Male""","""40-49""","[[""Yes"", ""No""], [""No"", ""No""], [""Yes"", ""No""]]",3


In [81]:
import random
import numpy as np
d = a.with_columns(pl.col("len").map_elements(lambda s: random.choices(range(1,9), k=s), return_dtype=pl.List(pl.Int32)).alias("weights"))
d = d.with_columns(pl.lit(np.array(random.choices(range(1,5), k=d.height))).alias("to_sample"))
d

Sex,AgeGroup,test,len,weights,to_sample
str,str,list[list[str]],u32,list[i32],i32
"""Male""","""30-39""","[[""No"", ""Yes""]]",1,[8],2
"""Female""","""30-39""","[[""Yes"", ""No""], [""No"", ""Yes""]]",2,"[5, 3]",4
"""Male""","""20-29""","[[""No"", ""Yes""], [""No"", ""Yes""]]",2,"[2, 7]",2
"""Female""","""20-29""","[[""Yes"", ""Yes""], [""Yes"", ""Yes""]]",2,"[7, 2]",4
"""Male""","""40-49""","[[""Yes"", ""No""], [""No"", ""No""], [""Yes"", ""No""]]",3,"[2, 7, 1]",3


In [104]:
np.random.choice(a=["sa"], size=1, p=[x/sum([8]) for x in [8]], replace=True)

array(['sa'], dtype='<U2')

In [114]:
from numpy.random import choice

def f(r):
    result = choice(a=range(0, len(r["test"])), size=r["to_sample"], p=[x/sum(r["weights"]) for x in r["weights"]], replace=True)
    return [r["test"][x] for x in result]
draw = d.with_columns(pl.struct(pl.col(["test", "weights", "to_sample"])).map_elements(f, return_dtype=pl.List(pl.List(str))).alias("draw"))

draw

Sex,AgeGroup,test,len,weights,to_sample,draw
str,str,list[list[str]],u32,list[i32],i32,list[list[str]]
"""Male""","""30-39""","[[""No"", ""Yes""]]",1,[8],2,"[[""No"", ""Yes""], [""No"", ""Yes""]]"
"""Female""","""30-39""","[[""Yes"", ""No""], [""No"", ""Yes""]]",2,"[5, 3]",4,"[[""No"", ""Yes""], [""Yes"", ""No""], … [""Yes"", ""No""]]"
"""Male""","""20-29""","[[""No"", ""Yes""], [""No"", ""Yes""]]",2,"[2, 7]",2,"[[""No"", ""Yes""], [""No"", ""Yes""]]"
"""Female""","""20-29""","[[""Yes"", ""Yes""], [""Yes"", ""Yes""]]",2,"[7, 2]",4,"[[""Yes"", ""Yes""], [""Yes"", ""Yes""], … [""Yes"", ""Yes""]]"
"""Male""","""40-49""","[[""Yes"", ""No""], [""No"", ""No""], [""Yes"", ""No""]]",3,"[2, 7, 1]",3,"[[""Yes"", ""No""], [""No"", ""No""], [""No"", ""No""]]"


In [121]:
mm = draw.explode("draw")

In [122]:
mm.with_columns(pl.col('draw').list.to_struct(
            fields=lambda idx: f"numbers_{idx}",
            n_field_strategy='max_width'
            )
        ).unnest('draw')

Sex,AgeGroup,test,len,weights,to_sample,numbers_0,numbers_1
str,str,list[list[str]],u32,list[i32],i32,str,str
"""Male""","""30-39""","[[""No"", ""Yes""]]",1,[8],2,"""No""","""Yes"""
"""Male""","""30-39""","[[""No"", ""Yes""]]",1,[8],2,"""No""","""Yes"""
"""Female""","""30-39""","[[""Yes"", ""No""], [""No"", ""Yes""]]",2,"[5, 3]",4,"""No""","""Yes"""
"""Female""","""30-39""","[[""Yes"", ""No""], [""No"", ""Yes""]]",2,"[5, 3]",4,"""Yes""","""No"""
"""Female""","""30-39""","[[""Yes"", ""No""], [""No"", ""Yes""]]",2,"[5, 3]",4,"""Yes""","""No"""
…,…,…,…,…,…,…,…
"""Female""","""20-29""","[[""Yes"", ""Yes""], [""Yes"", ""Yes""]]",2,"[7, 2]",4,"""Yes""","""Yes"""
"""Female""","""20-29""","[[""Yes"", ""Yes""], [""Yes"", ""Yes""]]",2,"[7, 2]",4,"""Yes""","""Yes"""
"""Male""","""40-49""","[[""Yes"", ""No""], [""No"", ""No""], [""Yes"", ""No""]]",3,"[2, 7, 1]",3,"""Yes""","""No"""
